# E91 Implementation

Rory Schadler, Blake Danziger  
Physics 75  
Lab 2  

## Protocol Steps
1. For each bit of the message, Alice and Bob share an EPR pair.
2. Alice picks a random measurement basis from a = {0, pi/4, pi/2} and Bob picks a random measurement basis from b= {pi/4, pi/2, 3pi/4}. Alice and Bob each measure their piece of the EPR pair in their randomly chosen measurement basis
3. Alice and Bob anounce the measurement bases they used for each shared pair.
4. Alice and Bob announce the results of their measurements for the pairs where their measurement bases did not coincide.

## Implementation Steps
1. Create $n$ bell pairs, one for each message bit.
2. choose measurement bases for Alice, Bob
    * rotate $|0\rangle$, $|1\rangle$ by $a \in \{0,\frac{\pi}{4},\frac{\pi}{2}\}$ or $b \in \{\frac{\pi}{4},\frac{\pi}{2},\frac{3\pi}{4}\}$*
3. Alice and Bob each measure their bits
4. Find pairs where Alice and Bob used the same measurement basis
5. Compare measurements of bits where Alice's and Bob's measurements did not match

Source: [https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661)

In [96]:
import qiskit
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.visualization import plot_histogram
from qiskit import *
from numpy.random import randint#, choice
import numpy as np
print("Imports Successful")

Imports Successful


In [97]:
msg_len = 10

alice_bases = [np.pi/4, np.pi/8, 0]
bob_bases = [np.pi/8, 0, -np.pi/8]

In [98]:
def alice_measure(qc):
    chosen_bases = []
    for i in range(msg_len):
        alice_q = 2*i
        base = randint(3)
        chosen_bases.append(base)
        rot = alice_bases[base]
        qc.u3(-1 * rot, 0, 0, alice_q)
        qc.measure(alice_q, alice_q)
#         qc.u3(rot, 0, 0, alice_q)
    return chosen_bases

def bob_measure(qc):
    chosen_bases = []
    for i in range(msg_len):
        bob_q = 2*i + 1
        base = randint(3)
        chosen_bases.append(base)
        rot = bob_bases[base]
        qc.u3(-1 * rot, 0, 0, bob_q)
#         qc.u3(-1 * rot, 0, 0, bob_q)
        qc.measure(bob_q,bob_q)
#         qc.u3(rot, 0, 0, bob_q)
    return chosen_bases

# help(qiskit.circuit.library.U3Gate)

In [99]:
# Prepare Bell States
def prepare():
    
    qc = QuantumCircuit(2*msg_len, 2*msg_len)
    for i in range(msg_len):
        q1 = 2*i
        q2 = 2*i + 1
        qc.x(q2)
        qc.h(q1)
        qc.cx(q1,q2)
    qc.barrier()
    qc.draw()
    
    return qc

qc = prepare()

In [100]:
# add measurements with random bases
def add_measurements(qc):
    a_bases = alice_measure(qc)
    b_bases = bob_measure(qc)
    
    return (a_bases, b_bases, qc)

(a_bases, b_bases, qc) = add_measurements(qc)
qc.draw()

┌───┐      ░ ┌───────────────┐┌─┐                                       »
 q_0: ┤ H ├──■───░─┤ U3(-pi/8,0,0) ├┤M├───────────────────────────────────────»
      ├───┤┌─┴─┐ ░ ├───────────────┤└╥┘                           ┌─┐         »
 q_1: ┤ X ├┤ X ├─░─┤ U3(-pi/8,0,0) ├─╫────────────────────────────┤M├─────────»
      ├───┤└───┘ ░ └─┬───────────┬─┘ ║ ┌─┐                        └╥┘         »
 q_2: ┤ H ├──■───░───┤ U3(0,0,0) ├───╫─┤M├─────────────────────────╫──────────»
      ├───┤┌─┴─┐ ░   ├───────────┤   ║ └╥┘                         ║ ┌─┐      »
 q_3: ┤ X ├┤ X ├─░───┤ U3(0,0,0) ├───╫──╫──────────────────────────╫─┤M├──────»
      ├───┤└───┘ ░   ├───────────┤   ║  ║ ┌─┐                      ║ └╥┘      »
 q_4: ┤ H ├──■───░───┤ U3(0,0,0) ├───╫──╫─┤M├──────────────────────╫──╫───────»
      ├───┤┌─┴─┐ ░ ┌─┴───────────┴─┐ ║  ║ └╥┘                      ║  ║ ┌─┐   »
 q_5: ┤ X ├┤ X ├─░─┤ U3(-pi/8,0,0) ├─╫──╫──╫───────────────────────╫──╫─┤M├───»
      ├───┤└───┘ ░ ├───────────────┤ ║  ║  ║ ┌─┐                   ║  ║ └╥┘   »
 q_6: ┤ H ├──■───░─┤ U3(-pi/4,0,0) ├─╫──╫──╫─┤M├───────────────────╫──╫──╫────»
      ├───┤┌─┴─┐ ░ └─┬───────────┬─┘ ║  ║  ║ └╥┘                   ║  ║  ║ ┌─┐»
 q_7: ┤ X ├┤ X ├─░───┤ U3(0,0,0) ├───╫──╫──╫──╫────────────────────╫──╫──╫─┤M├»
      ├───┤└───┘ ░ ┌─┴───────────┴─┐ ║  ║  ║  ║ ┌─┐                ║  ║  ║ └╥┘»
 q_8: ┤ H ├──■───░─┤ U3(-pi/4,0,0) ├─╫──╫──╫──╫─┤M├────────────────╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ ├───────────────┤ ║  ║  ║  ║ └╥┘                ║  ║  ║  ║ »
 q_9: ┤ X ├┤ X ├─░─┤ U3(-pi/8,0,0) ├─╫──╫──╫──╫──╫─────────────────╫──╫──╫──╫─»
      ├───┤└───┘ ░ ├───────────────┤ ║  ║  ║  ║  ║ ┌─┐             ║  ║  ║  ║ »
q_10: ┤ H ├──■───░─┤ U3(-pi/8,0,0) ├─╫──╫──╫──╫──╫─┤M├─────────────╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ └─┬───────────┬─┘ ║  ║  ║  ║  ║ └╥┘             ║  ║  ║  ║ »
q_11: ┤ X ├┤ X ├─░───┤ U3(0,0,0) ├───╫──╫──╫──╫──╫──╫──────────────╫──╫──╫──╫─»
      ├───┤└───┘ ░ ┌─┴───────────┴─┐ ║  ║  ║  ║  ║  ║ ┌─┐          ║  ║  ║  ║ »
q_12: ┤ H ├──■───░─┤ U3(-pi/8,0,0) ├─╫──╫──╫──╫──╫──╫─┤M├──────────╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ └─┬───────────┬─┘ ║  ║  ║  ║  ║  ║ └╥┘          ║  ║  ║  ║ »
q_13: ┤ X ├┤ X ├─░───┤ U3(0,0,0) ├───╫──╫──╫──╫──╫──╫──╫───────────╫──╫──╫──╫─»
      ├───┤└───┘ ░   ├───────────┤   ║  ║  ║  ║  ║  ║  ║ ┌─┐       ║  ║  ║  ║ »
q_14: ┤ H ├──■───░───┤ U3(0,0,0) ├───╫──╫──╫──╫──╫──╫──╫─┤M├───────╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ ┌─┴───────────┴─┐ ║  ║  ║  ║  ║  ║  ║ └╥┘       ║  ║  ║  ║ »
q_15: ┤ X ├┤ X ├─░─┤ U3(-pi/8,0,0) ├─╫──╫──╫──╫──╫──╫──╫──╫────────╫──╫──╫──╫─»
      ├───┤└───┘ ░ ├───────────────┤ ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐    ║  ║  ║  ║ »
q_16: ┤ H ├──■───░─┤ U3(-pi/4,0,0) ├─╫──╫──╫──╫──╫──╫──╫──╫─┤M├────╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ └┬──────────────┤ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘    ║  ║  ║  ║ »
q_17: ┤ X ├┤ X ├─░──┤ U3(pi/8,0,0) ├─╫──╫──╫──╫──╫──╫──╫──╫──╫─────╫──╫──╫──╫─»
      ├───┤└───┘ ░ ┌┴──────────────┤ ║  ║  ║  ║  ║  ║  ║  ║  ║ ┌─┐ ║  ║  ║  ║ »
q_18: ┤ H ├──■───░─┤ U3(-pi/4,0,0) ├─╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─╫──╫──╫──╫─»
      ├───┤┌─┴─┐ ░ └┬──────────────┤ ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘ ║  ║  ║  ║ »
q_19: ┤ X ├┤ X ├─░──┤ U3(pi/8,0,0) ├─╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
      └───┘└───┘ ░  └──────────────┘ ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
 c_0: ═══════════════════════════════╩══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬══╬═»
                                        ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
 c_1: ══════════════════════════════════╬══╬══╬══╬══╬══╬══╬══╬══╬══╩══╬══╬══╬═»
                                        ║  ║  ║  ║  ║  ║  ║  ║  ║     ║  ║  ║ »
 c_2: ══════════════════════════════════╩══╬══╬══╬══╬══╬══╬══╬══╬═════╬══╬══╬═»
                                           ║  ║  ║  ║  ║  ║  ║  ║     ║  ║  ║ »
 c_3: ═════════════════════════════════════╬══╬══╬══╬══╬══╬══╬══╬═════╩══╬══╬═»
                                           ║  ║  ║  ║  ║  ║  ║  ║        ║  ║ »
 c_4: ═════════════════════════════════════╩══╬══╬══╬══╬══╬══╬══╬════════╬══╬═»
      

In [101]:
qasm_sim = Aer.get_backend('qasm_simulator')
qobj = assemble(qc, shots=1, memory=True)
result = qasm_sim.run(qobj).result()
# get_counts returns a dict, with the classical register as the *key* 
# which feels silly. this ugly idiom retrieves that key. surely a 
# better way to do this
measurements = [k for k in result.get_counts()][0]

alice_m = ""
bob_m = ""
for i in range(2*msg_len):
    if not i%2:
        alice_m += measurements[i]
    else:
        bob_m += measurements[i]

print(alice_m)
print(bob_m)
print(measurements)
print(a_bases)
print(b_bases)

0000001010
1111111101
01010101010111011001
[1, 2, 2, 0, 0, 1, 1, 2, 0, 0]
[0, 1, 0, 1, 0, 1, 1, 0, 2, 2]


In [102]:
for idx, (a_b, b_b) in enumerate(zip(a_bases, b_bases)):
    if((a_b == 1 and b_b == 0) or (a_b == 2 and b_b == 1)):
        a_m = alice_m[idx]
        b_m = bob_m[idx]
        print("match! Alice measured {} and Bob measured {} on qubit {}".format(a_m, b_m, idx))

match! Alice measured 0 and Bob measured 1 on qubit 0
match! Alice measured 0 and Bob measured 1 on qubit 1


In [103]:
def test1():
    qc = prepare()
    (a_bases, b_bases, qc) = add_measurements(qc)
    qasm_sim = Aer.get_backend('qasm_simulator')

    qobj = assemble(qc, shots=1, memory=True)
    result = qasm_sim.run(qobj).result()
    # get_counts returns a dict, with the classical register as the *key* 
    # which feels silly. this ugly idiom retrieves that key. surely a 
    # better way to do this
    measurements = [k for k in result.get_counts()][0]

    alice_m = ""
    bob_m = ""
    for i in range(2*msg_len):
        if not i%2:
            alice_m += measurements[i]
        else:
            bob_m += measurements[i]

    print(alice_m)
    print(bob_m)
    print(measurements)
    print(a_bases)
    print(b_bases)

    for idx, (a_b, b_b) in enumerate(zip(a_bases, b_bases)):
        if((a_b == 1 and b_b == 0) or (a_b == 2 and b_b == 1)):
            a_m = alice_m[idx]
            b_m = bob_m[idx]
            print("match! Alice measured {} and Bob measured {} on qubit {}".format(a_m, b_m, idx))
            
            
test1()
test1()
test1()
test1()
test1()


1010000010
0101101101
10011001010001011001
[0, 1, 1, 1, 1, 0, 0, 1, 1, 0]
[0, 2, 0, 0, 1, 0, 2, 2, 2, 1]
match! Alice measured 1 and Bob measured 0 on qubit 2
match! Alice measured 0 and Bob measured 1 on qubit 3
1100011110
0010100001
10100100011010101001
[1, 0, 1, 0, 2, 2, 2, 2, 0, 2]
[1, 2, 1, 1, 0, 0, 0, 2, 2, 1]
match! Alice measured 0 and Bob measured 1 on qubit 9
1111111110
0000000001
10101010101010101001
[2, 2, 1, 1, 2, 2, 0, 1, 0, 2]
[0, 1, 0, 1, 2, 1, 0, 1, 1, 0]
match! Alice measured 1 and Bob measured 0 on qubit 1
match! Alice measured 1 and Bob measured 0 on qubit 2
match! Alice measured 1 and Bob measured 0 on qubit 5
1110000001
0001111010
10101001010101000110
[1, 1, 1, 2, 1, 2, 1, 2, 0, 1]
[0, 1, 0, 0, 2, 0, 2, 0, 1, 0]
match! Alice measured 1 and Bob measured 0 on qubit 0
match! Alice measured 1 and Bob measured 0 on qubit 2
match! Alice measured 1 and Bob measured 0 on qubit 9
0111111100
1000100011
01101010111010100101
[0, 2, 1, 1, 2, 1, 0, 0, 0, 0]
[2, 1, 0, 1, 2, 0, 1